## Задача классификации на примере определения
## для присланной научной статьи рубрики журнала.

Есть журнал трудов Крыловского научного центра https://transactions-ksrc.ru/ из области судостроения, авторы присылают свои научные статьи. Есть задачи по определению в какую рубрику отнести статью, а также определение ключевых слов по присланной статье в формате docx. В данном ноутбуке идет подготовка датасета и решение для задачи классификации.

In [1]:
from docx import Document
import pandas as pd
import numpy as np
import os
import re
import nltk
from sklearn.model_selection import train_test_split
from IPython.core.display import display, Image
from collections import defaultdict

In [2]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mihachev_ev\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

In [7]:
#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

In [8]:
#Preprocess function
def preprocess_text(text):
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [6]:
articles = pd.DataFrame(columns=['text', 'section', 'keywords'])
articles.head()

,text,section,keywords


In [7]:
# Для обучения использую уже готовые статьи журнала, в которых есть лишняя информация, и которой точно нет в новых статьях
WORDS_TO_DEL = ['doi', 'удк', 'udc', 'теория корабля и строительная механика', 'naval architecture','судовые энергетические установки и электроэнергетика', 
                'проектирование и конструкция судов', 'ship design and structure', 'физические поля корабля', 
                'ship powering and electric generation systems', 'другие вопросы морской техники', 
                'miscellaneous', 'ship signatures' ]
#KEYW_P = ['Ключевые слова:', 'Keywords:']

In [17]:
# статьи разбиты по каталогом, которые соответсвуют разным рубрикам журнала.
articles = pd.DataFrame(columns=['text', 'section'])
for sect in range(1, 6):
    path = ('./word/'+str(sect)+'/')
    num = 0
    for filename in os.listdir(path):
        f = open(path + filename, 'rb')
        print(f)
        try:
            document = Document(f)
            text = ''
            keyw = ''
        
            for p in document.paragraphs:
                                    
                #print(p.text)
                if p.text ==None or p.text=="": 
                    pass
                elif any(kws in p.text.lower() for kws in KEYW_P):  
                    keyw += ' ' + p.text
                #elif  re.compile('|'.join(WORDS_TO_DEL),re.IGNORECASE).search(p.text):
                elif any(ele in p.text.lower() for ele in WORDS_TO_DEL):  #any(WORDS_TO_DEL) in p.text:
                    continue
                
                    
                else: 
                    text = text + ' ' + p.text
        except:
            pass
        #print(text)
        #text = text.replace(u'\xa0', u' ')
        #text = text.replace('\n', u' ')
        
        text = preprocess_text(text)
        #print(text)
        article = pd.DataFrame([[text, sect, keyw]], columns=['text', 'section', 'keywords'])
       
        articles = pd.concat([articles, article], ignore_index=False)
        num += 1
        #if num > 2:
            #f.close()
            #break
        f.close()
articles = articles.reset_index(drop=True)  
articles

<_io.BufferedReader name='./word/1/01_Вишневский_ВЕРСТКА.docx'>
<_io.BufferedReader name='./word/1/01_Галушина_Сверчков_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/01_Пустошный_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/01_Пустошный_правка.docx'>
<_io.BufferedReader name='./word/1/01_Сазонов_Добродеев_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/02_Бахарев_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/02_Добродеев_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/02_Косоротов_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/02_Нигматуллин_Рыбакина_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/02_Сверчков_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/03_Крыжевич_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/03_Орлов_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/03_Соколов_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/03_Федонюк_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./word/1/03_Яковлев_ВЕРСТКА+.docx'>
<_io.BufferedReader name='./w

,text,section,keywords
0,л вишневский н половинкин р тогуняц фгуп крыло...,1,
1,м галушин сверчок чичерин д фомичев л г щемели...,1,
2,пустошный г дарчиев г фролова фгуп крыловский ...,1,
3,пустошный сверчок п шевцов фгуп крыловский гос...,1,
4,добродеев е сазонов фгуп крыловский государств...,1,
...,...,...,...
109,э алексеев е бабчук крулев фгуп крыловский гос...,5,
110,л ковтун н л ковтун т харитоненко н шарков фгу...,5,
111,н л ковтун фгуп крыловский государственный нау...,5,
112,шлячок б разлетов м м карулина н вальдман беля...,5,


In [18]:
articles['text'][0]

'л вишневский н половинкин р тогуняц фгуп крыловский государственный научный центр санкт петербург россия ао гипрорыбфлот санкт петербург россия винт изменять шаг основной результат направление исследование объект цель научный работа проводить обзор результат исследование получать последний год область нетрадиционный движитель винт изменять шаг материал метод использовать новый подход создание расчетный метод экспериментальный средство основной результат показывать многий случай применение движитель связанный изменение условие плавание судно мочь иметь преимущество традиционный движитель заключение представлять материал ориентировать исследователь продолжение работа позволять уточнять область использование данные движитель ключевой слово гребной винт винт изменять шаг динамический отключение лопасть ступица автор заявлять отсутствие возможный конфликт интерес l vishnevsky v polovinkin a togunyats krylov state research centre st petersburg russia jsc giprorybflot st petersburg russia ad

In [19]:
#articles.to_csv('articles.csv', encoding='utf-8', index=False)

In [10]:
articles = pd.read_csv('articles.csv')
articles

,text,section,keywords
0,л вишневский н половинкин р тогуняц фгуп крыло...,1,NaN
1,м галушин сверчок чичерин д фомичев л г щемели...,1,NaN
2,пустошный г дарчиев г фролова фгуп крыловский ...,1,NaN
3,пустошный сверчок п шевцов фгуп крыловский гос...,1,NaN
4,добродеев е сазонов фгуп крыловский государств...,1,NaN
...,...,...,...
109,э алексеев е бабчук крулев фгуп крыловский гос...,5,NaN
110,л ковтун н л ковтун т харитоненко н шарков фгу...,5,NaN
111,н л ковтун фгуп крыловский государственный нау...,5,NaN
112,шлячок б разлетов м м карулина н вальдман беля...,5,NaN


In [7]:
articles['text'][0]

'л вишневский н половинкин р тогуняц фгуп крыловский государственный научный центр санкт петербург россия ао гипрорыбфлот санкт петербург россия винт изменять шаг основной результат направление исследование объект цель научный работа проводить обзор результат исследование получать последний год область нетрадиционный движитель винт изменять шаг материал метод использовать новый подход создание расчетный метод экспериментальный средство основной результат показывать многий случай применение движитель связанный изменение условие плавание судно мочь иметь преимущество традиционный движитель заключение представлять материал ориентировать исследователь продолжение работа позволять уточнять область использование данные движитель ключевой слово гребной винт винт изменять шаг динамический отключение лопасть ступица автор заявлять отсутствие возможный конфликт интерес l vishnevsky v polovinkin a togunyats krylov state research centre st petersburg russia jsc giprorybflot st petersburg russia ad

In [11]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      114 non-null    object 
 1   section   114 non-null    int64  
 2   keywords  0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.8+ KB


In [12]:
tokenizer = nltk.RegexpTokenizer(r"[a-zа-яё]+")

def text2tokens(text, num=1000):
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    tokens = tokenizer.tokenize(text)
    if len(tokens) > num:
        return tokens[:num]
    else:
        return ['#'] * (num - len(tokens)) +  tokens




In [9]:
text2tokens(articles.iloc[1]["text"])

['галушин',
 'сверчок',
 'чичерин',
 'фомичев',
 'щемелинин',
 'кок',
 'фгуп',
 'крыловский',
 'государственный',
 'научный',
 'центр',
 'санкт',
 'петербург',
 'россия',
 'navalprogetti',
 'триест',
 'италия',
 'гидродинамический',
 'характеристика',
 'судно',
 'искусственный',
 'каверна',
 'движение',
 'ограниченный',
 'фарватер',
 'объект',
 'цель',
 'научный',
 'работа',
 'объект',
 'исследование',
 'являться',
 'судно',
 'контейнеровоз',
 'внутренний',
 'плавание',
 'увеличивать',
 'коэффициент',
 'общий',
 'полнота',
 'главный',
 'размерение',
 'корпус',
 'судно',
 'удовлетворять',
 'требование',
 'эксплуатация',
 'западноевропейский',
 'внутренний',
 'водный',
 'путь',
 'цель',
 'работа',
 'состоять',
 'экспериментальный',
 'проверка',
 'эффективность',
 'применение',
 'искусственный',
 'каверна',
 'снижение',
 'гидродинамический',
 'сопротивление',
 'различный',
 'эксплуат',
 'ционный',
 'условие',
 'также',
 'определение',
 'влияние',
 'каверна',
 'характеристика',
 'маневренн

In [10]:
articles.text

0      л вишневский н половинкин р тогуняц фгуп крыло...
1      м галушин сверчок чичерин д фомичев л г щемели...
2      пустошный г дарчиев г фролова фгуп крыловский ...
3      пустошный сверчок п шевцов фгуп крыловский гос...
4      добродеев е сазонов фгуп крыловский государств...
                             ...                        
109    э алексеев е бабчук крулев фгуп крыловский гос...
110    л ковтун н л ковтун т харитоненко н шарков фгу...
111    н л ковтун фгуп крыловский государственный нау...
112    шлячок б разлетов м м карулина н вальдман беля...
113    м власьев фгуп крыловский государственный науч...
Name: text, Length: 114, dtype: object

In [13]:
token_ids = defaultdict(lambda: len(token_ids))

def text2tokenIds(text, num=1000):
    tokens = text2tokens(text, num)
    result = []
    for token in tokens:
        result.append(token_ids[token])
    return np.array(result)

text2tokenIds(articles.iloc[0]["text"])


array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         8,   9,  10,  13,  14,  15,  16,  17,  18,  19,  20,  21,   6,
        22,  23,  24,  17,  19,  25,  26,  27,  28,  29,  30,  13,  14,
        15,  31,  32,  33,  34,  35,  36,  37,  32,  38,  39,  16,  17,
        40,  41,  42,  43,  30,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  30,  53,  54,  31,  55,  56,  57,  22,  58,  59,  28,  60,
        61,  30,  62,  63,  64,  13,  13,  14,  15,  65,  66,  67,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,
        82,  83,  84,  85,  86,  82,  83,  84,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  92,  97,  94,  80,  98,  99, 100, 101,
        95, 102, 103, 104,  89, 105, 106,  92, 107,  98, 108, 109, 110,
       111, 112, 113, 114,  92, 115, 116, 117, 118,  94, 119, 107,  90,
        91, 120, 109, 121, 122, 123,  94,  87,  88,  89, 109, 124, 125,
       126, 127, 128,  94, 129, 130, 131, 132,  89, 133, 134, 13

In [14]:
articles.section

0      1
1      1
2      1
3      1
4      1
      ..
109    5
110    5
111    5
112    5
113    5
Name: section, Length: 114, dtype: int64

In [15]:
max_len = 1000
data = articles.text.apply(lambda txt: text2tokenIds(txt, max_len))  
X = pd.DataFrame(data, index = articles.index)
y = articles.section

max_features = len(token_ids)

In [53]:
articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114 entries, 0 to 113
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   text      114 non-null    object 
 1   section   114 non-null    int64  
 2   keywords  0 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.8+ KB


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=96622)

In [17]:
X_train = np.asarray(X_train)
X_test   = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test   = np.asarray(y_test)

X_train = np.expand_dims(X_train, -1)
X_test   = np.expand_dims(X_test, -1)
y_train = np.expand_dims(y_train, -1)
y_test   = np.expand_dims(y_test, -1)

In [18]:
X_train

array([[[array([2594,    3,    4,    5,    6,    7,    8,    9,   10,  435, 1280,
        848, 1033, 2485, 2564, 2595,  518, 1497,   20,   21,    6,   22,
         20,   22,  304,  848, 1033,  521, 2485, 2564, 2595,  518, 1497,
       2596,  210, 2597,  509,  982, 2598,  348,   21,   22,  286,  445,
        848,   43, 1113,  320,  552,  727,   31,   32,  470, 2599, 2600,
        848, 1033,  521, 2485, 2564, 2595,  518, 1497,  464,  554,  555,
          4,    5,    6,    7,  212,  463,  982, 1033,  383, 1110, 2578,
        554,  555, 2579,  726, 1731,  498,  253,  848, 1524, 1873,  435,
        498, 1999,  228,   25, 2601,  772, 2561,  521,  263, 1141, 1980,
       1653, 1030, 2602, 2603,  210,  848,   16,   17,   40,  488, 1134,
        848, 2604,   60,  444, 1288, 2599,  848, 1113,  521,  263, 1980,
       1653, 1030, 2602, 2603,  818,   17, 2605,  474,  383, 2598,  348,
       1271, 2599, 1033,  570, 2578, 2579,   53,   60,  210,  848,  478,
        818,  521,  263, 1141, 2606,  982,

In [19]:
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM

evaluate_samples = 1000
attempt = 0
lstm_hidden_vars = 16
embedding_size = 32
#max_len = 1200
max_features = 500

model = Sequential()
model.add(Embedding(max_features, embedding_size)) #, input_length=max_len
model.add(LSTM(lstm_hidden_vars, return_sequences=True))
model.add(LSTM(lstm_hidden_vars))

model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))


model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

In [20]:
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(512, dropout=0.2, recurrent_dropout=0.3, return_sequences=False))
#model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'], learning_rate = 0.001)

In [35]:
y_test

array([[2],
       [1],
       [2],
       [2],
       [4],
       [1],
       [2],
       [4],
       [1],
       [4],
       [2],
       [2],
       [2],
       [1],
       [1],
       [2],
       [1],
       [1],
       [2],
       [2],
       [1],
       [1],
       [1],
       [1],
       [1],
       [4],
       [3],
       [5],
       [5]], dtype=int64)

In [34]:
X_train

array([[[array([2594,    3,    4,    5,    6,    7,    8,    9,   10,  435, 1280,
        848, 1033, 2485, 2564, 2595,  518, 1497,   20,   21,    6,   22,
         20,   22,  304,  848, 1033,  521, 2485, 2564, 2595,  518, 1497,
       2596,  210, 2597,  509,  982, 2598,  348,   21,   22,  286,  445,
        848,   43, 1113,  320,  552,  727,   31,   32,  470, 2599, 2600,
        848, 1033,  521, 2485, 2564, 2595,  518, 1497,  464,  554,  555,
          4,    5,    6,    7,  212,  463,  982, 1033,  383, 1110, 2578,
        554,  555, 2579,  726, 1731,  498,  253,  848, 1524, 1873,  435,
        498, 1999,  228,   25, 2601,  772, 2561,  521,  263, 1141, 1980,
       1653, 1030, 2602, 2603,  210,  848,   16,   17,   40,  488, 1134,
        848, 2604,   60,  444, 1288, 2599,  848, 1113,  521,  263, 1980,
       1653, 1030, 2602, 2603,  818,   17, 2605,  474,  383, 2598,  348,
       1271, 2599, 1033,  570, 2578, 2579,   53,   60,  210,  848,  478,
        818,  521,  263, 1141, 2606,  982,

In [21]:
batch_size  = 8
epochs = 2

model.fit(
    X_train, y_train, 
    batch_size=batch_size, 
    validation_data=(X_test, y_test),
    workers=1, use_multiprocessing=False,
    epochs=epochs
)

ValueError: Error when checking input: expected embedding_1_input to have 2 dimensions, but got array with shape (85, 1, 1)

In [38]:
batch_size  = 8
epochs = 2

model.fit(
    X_train.values, y_train.values, 
    batch_size=batch_size, 
    validation_data=(X_test.values, y_test.values),
    workers=1, use_multiprocessing=False,
    epochs=epochs
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
#docs = docs.tolist()
#create a vocabulary of words, 
cv=CountVectorizer(max_df=0.95,         # ignore words that appear in 95% of documents
                   max_features=10000,  # the size of the vocabulary
                   ngram_range=(1,3)    # vocabulary contains single words, bigrams, trigrams
                  )
word_count_vector=cv.fit_transform(articles['text'])

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

NameError: name 'word_count_vector' is not defined

In [39]:
from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics

Using TensorFlow backend.


In [40]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

In [41]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [42]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
X_train = newsgroups_train.data
X_test = newsgroups_test.data
y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [48]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("C:\\Users\\kamran\\Documents\\GitHub\\RMDL\\Examples\\Glove\\glove.6B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)

In [49]:
def Build_Model_RNN_Text(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    """
    def buildModel_RNN(word_index, embeddings_index, nclasses,  MAX_SEQUENCE_LENGTH=500, EMBEDDING_DIM=50, dropout=0.5):
    word_index in word index ,
    embeddings_index is embeddings index, look at data_helper.py
    nClasses is number of classes,
    MAX_SEQUENCE_LENGTH is maximum lenght of text sequences
    """

    model = Sequential()
    hidden_layer = 3
    gru_node = 32

    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            if len(embedding_matrix[i]) != len(embedding_vector):
                print("could not broadcast input array from shape", str(len(embedding_matrix[i])),
                      "into shape", str(len(embedding_vector)), " Please make sure your"
                                                                " EMBEDDING_DIM is equal to embedding_vector file ,GloVe,")
                exit(1)
            embedding_matrix[i] = embedding_vector
    model.add(Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=True))


    print(gru_node)
    for i in range(0,hidden_layer):
        model.add(GRU(gru_node,return_sequences=True, recurrent_dropout=0.2))
        model.add(Dropout(dropout))
    model.add(GRU(gru_node, recurrent_dropout=0.2))
    model.add(Dropout(dropout))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))


    model.compile(loss='sparse_categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
    return model

In [58]:
from keras.preprocessing.text import Tokenizer
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
X_train = newsgroups_train.data
X_test = newsgroups_test.data
y_train = newsgroups_train.target
y_test = newsgroups_test.target

X_train_Glove,X_test_Glove, word_index,embeddings_index = loadData_Tokenizer(X_train,X_test)


model_RNN = Build_Model_RNN_Text(word_index,embeddings_index, 20)

model_RNN.fit(X_train_Glove, y_train,
                              validation_data=(X_test_Glove, y_test),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_RNN.predict_classes(X_test_Glove)

print(metrics.classification_report(y_test, predicted))

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [45]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 20)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_DNN.predict(X_test_tfidf)

print(metrics.classification_report(y_test, predicted))

MemoryError: 

In [44]:
def loadData_Tokenizer(X_train, X_test,MAX_NB_WORDS=75000,MAX_SEQUENCE_LENGTH=500):
    np.random.seed(7)
    text = np.concatenate((X_train, X_test), axis=0)
    text = np.array(text)
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(text)
    sequences = tokenizer.texts_to_sequences(text)
    word_index = tokenizer.word_index
    text = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    print('Found %s unique tokens.' % len(word_index))
    indices = np.arange(text.shape[0])
    # np.random.shuffle(indices)
    text = text[indices]
    print(text.shape)
    X_train = text[0:len(X_train), ]
    X_test = text[len(X_train):, ]
    embeddings_index = {}
    f = open("C:\\Users\\kamran\\Documents\\GitHub\\RMDL\\Examples\\Glove\\glove.6B.50d.txt", encoding="utf8")
    for line in f:

        values = line.split()
        word = values[0]
        try:
            coefs = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = coefs
    f.close()
    print('Total %s word vectors.' % len(embeddings_index))
    return (X_train, X_test, word_index,embeddings_index)